# Preprocess - Prepare_data
This notebook is used to prepare the data. For both training as predicting the same pipeline is used, but with a slight difference in the resulting output (with/without target variable). For The goal is a Xy-dataset in parquet format, with:
* For training:
    - The number of WMO-clients is the target-variable y. 
    - A number of columns with features
    - Index contains a region and time interval
    - Scaled / normalised according to type of variable
    - Filename contains a datetime suffix
* For predicting:
    - A number of columns with features
    - Index contains a region and time interval
    - Scaled / normalised according to type of variable
    - Filename contains a datetime suffix

## Content
* **Imports**: Imports of needed Python packages
* **Settings**: Hard coded variables needed to collect data like sources, tablenames, columnnames, etc. 
* **Funtions**: Resuable functions
* **Load data**: Load data to prepare dataset
* **Transform / prepare dataset**: Combining all data to one table
* **Write result**: Writing result to '../data'
* **Appendix**: Usefull code to preserve
    * ...

## Requirements
The packages to be installed (besides standard Python packages) are:
* pandas >=1.1.5
* cbsodata >=1.3.3

# Imports

In [ ]:
from os import listdir
from os.path import isfile, join
import pandas as pd
import cbsodata
from datetime import datetime

from typing import Union
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline, make_pipeline

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

# Settings

In [ ]:
# IMPORTANT: Set the variables right below!
# True if prepare train dataset, False if prepare predict dataset
train=True 


# General settings:
# Location all data
datapath = '../data/'

DICT_REALTIVELY_COLS = {'aantalinwoners': ['percentagewmoclienten', 'mannen', 'vrouwen', 'k0tot15jaar', 'k15tot25jaar',
                                           'k25tot45jaar', 'k45tot65jaar', 'k65jaarofouder', 'ongehuwd', 'gehuwd', 'gescheiden',
                                           'verweduwd', 'westerstotaal', 'nietwesterstotaal', 'marokko', 
                                           'nederlandseantillenenaruba', 'suriname', 'turkije', 'overignietwesters', 
                                           'geboortetotaal', 'geboorterelatief', 'sterftetotaal', 'sterfterelatief', 
                                           'aantalinkomensontvangers', 'personenpersoortuitkeringbijstand', 
                                           'personenpersoortuitkeringao', 'personenpersoortuitkeringww', 
                                           'personenpersoortuitkeringaow'],
                        'huishoudenstotaal': ['eenpersoonshuishoudens', 'huishoudenszonderkinderen', 'huishoudensmetkinderen'],
                        'bedrijfsvestigingentotaal': ['alandbouwbosbouwenvisserij', 'bfnijverheidenenergie', 
                                                      'gihandelenhoreca', 'hjvervoerinformatieencommunicatie', 
                                                      'klfinancieledienstenonroerendgoed', 'mnzakelijkedienstverlening', 
                                                      'rucultuurrecreatieoverigediensten'],
                        'personenautostotaal': ['personenautosbrandstofbenzine', 'personenautosoverigebrandstof'],
                        'oppervlaktetotaal': ['oppervlakteland', 'oppervlaktewater']}
LIST_NORM_COLS = ['wmoclienten', 'aantalinwoners', 'huishoudenstotaal', 'gemiddeldehuishoudensgrootte', 'bevolkingsdichtheid', 
                  'woningvoorraad', 'gemiddeldewoningwaarde', 'percentageeengezinswoning', 'percentagemeergezinswoning', 
                  'percentagebewoond', 'percentageonbewoond', 'koopwoningen', 'huurwoningentotaal', 'inbezitwoningcorporatie', 
                  'inbezitoverigeverhuurders', 'eigendomonbekend', 'bouwjaarvoor2000', 'bouwjaarvanaf2000', 
                  'gemiddeldelektriciteitsverbruiktotaal', 'gemelectriciteitsverbruikappartement', 
                  'gemelectriciteitsverbruiktussenwoning', 'gemelectriciteitsverbruikhoekwoning', 
                  'gemelectriciteitsverbruiktweeondereenkapwoning', 'gemelectriciteitsverbruikvrijstaandewoning', 
                  'gemelectriciteitsverbruikhuurwoning', 'gemelectriciteitsverbruikeigenwoning', 
                  'gemiddeldaardgasverbruiktotaal', 'gemgasverbruikappartement', 'gemgasverbruiktussenwoning', 
                  'gemgasverbruikhoekwoning', 'gemgasverbruiktweeondereenkapwoning', 'gemgasverbruikvrijstaandewoning', 
                  'gemgasverbruikhuurwoning', 'gemgasverbruikeigenwoning', 'percentagewoningenmetstadsverwarming',
                  'gemiddeldinkomenperinkomensontvanger', 'gemiddeldinkomenperinwoner', 'k40personenmetlaagsteinkomen', 
                  'k20personenmethoogsteinkomen', 'actieven1575jaar', 'k40huishoudensmetlaagsteinkomen', 
                  'k20huishoudensmethoogsteinkomen', 'huishoudensmeteenlaaginkomen', 'huishonderofrondsociaalminimum',
                  'bedrijfsvestigingentotaal', 'personenautostotaal', 'motorfietsen', 'afstandtothuisartsenpraktijk', 
                  'afstandtotgrotesupermarkt', 'afstandtotkinderdagverblijf', 'afstandtotschool', 'scholenbinnen3km',
                  'oppervlaktetotaal', 'matevanstedelijkheid', 'omgevingsadressendichtheid']
# DROP_COLS = ['financieringsvorm', 'wmoclientenper1000inwoners', 'gemeentenaam', 'meestvoorkomendepostcode', 'dekkingspercentage', 'totaaldiefstaluitwoningschuured', 
#                'vernielingmisdrijftegenopenbareorde', 'geweldsenseksuelemisdrijven', 'personenautosjongerdan6jaar', 
#                'personenautos6jaarenouder', 'bedrijfsmotorvoertuigen']
DROP_COLS = ['financieringsvorm', 'gemeentenaam', 'meestvoorkomendepostcode', 'dekkingspercentage', 'totaaldiefstaluitwoningschuured', 
               'vernielingmisdrijftegenopenbareorde', 'geweldsenseksuelemisdrijven', 'personenautosjongerdan6jaar', 
               'personenautos6jaarenouder', 'bedrijfsmotorvoertuigen']

# Functions

In [ ]:
def get_latest_file(datapath='../data/', train=True):
    """
    Method to get the latest file to preprare
    
    :params str datapath: String with the (respectively) directory where the data can be found. Default = '../data'
    :params bool train: Boolean to indicate if expected dataframe should be for preparing training data. Default = True
    
    return: pd.DataFrame
    """
    # Get list with file
    onlyfiles = sorted([f for f in listdir(datapath) if isfile(join(datapath, f))])
    # Get last file
    if train:
        filename = [s for s in onlyfiles if "df_get_for_train_WMO" in s][-1]
    else:
        filename = [s for s in onlyfiles if "df_get_for_predict" in s][-1]
    # Get list with last files
    df = pd.read_parquet(datapath+filename)
    return df


class RelativeColumnScaler(BaseEstimator, TransformerMixin):
    """
    This is a transformer class to scale a (number of) column(s) based on another column.
    """

    def __init__(self, dict_relatively_cols=None):
        """
        :param dict(str:list[str]) dict_relatively_cols: Dictionary with the base column as key and as a value a list with one 
                                                         or more columnsnames that need to be transformed. 
        """
        self.dict_relatively_cols = dict_relatively_cols

    def fit(self, X, y=None):
        """
        Standard fit method of transformer (selects all columns in columns arg is None)

        :param pd.DataFrame X: Enables a DataFrame as input
        :param pd.Series y: Enables a target as input
        
        :return: return object itself
        """
        # nothing to fit here people, move along

        return self

    def transform(self, X) -> Union[pd.DataFrame, pd.Series]:
        """
        Standard transform method of transformer which scales the columns based on a base column.

        :param (pd.DataFrame) X: DataFrame to select and transform columns from
        
        :return: pd.DataFrame or pd.Series containing only the selected columns
        """
        assert isinstance(X, pd.DataFrame)
     
        try:
            for base_col, relatively_cols in self.dict_relatively_cols.items():
                X[relatively_cols] = X[relatively_cols].div(X[base_col], axis=0)
            return X
        except KeyError:
            colslist = [item for sublist in list(self.dict_relatively_cols.values()) for item in sublist]
            cols_error = list(set(colslist) - set(X.columns))
            raise KeyError("The DataFrame does not include the columns: %s" % cols_error)

            
class CustomScaler(BaseEstimator, TransformerMixin):
    """
    This is a transformer class to scale the selected columns using a defined scaler
    """

    def __init__(self, cols, scaler):
        """

        :param list[str] cols: List of columns to be selected.
        :param scaler: Scaler to apply, i.e. MinMaxScaler() from sklearn
        """
        self.cols = cols
        self.scaler = scaler

    def fit(self, X, y=None):
        """
        Standard fit method of transformer which fits the scaler to X

        :param pd.DataFrame X: DataFrame with the feature columns, including the column(s) to scale
        :param pd.Series y: Default None, not used in fit. The target values in a model
        :return: Fitted scaler for the selected column(s)
        """

        self.cols = [c for c in self.cols if c in X.columns]
        self.scaler.fit(X[self.cols])

        return self

    def transform(self, X):
        """
        Standard transform method of transformer which transforms the dataset with a scaler for the
        selected column(s)

        :param pd.DataFrame X: DataFrame with the feature columns, including the categorical column(s)
        :return: Transformed dataset X (with scaler as defined) for the selected column(s)
        """

        X = X.copy()
        X.loc[:, self.cols] = self.scaler.transform(X[self.cols])

        return X
            
    
# def make_cols_relatively(df, dict_relatively_cols):
#     for base_col, relatively_cols in dict_relatively_cols.items():
#         df[relatively_cols] = df[relatively_cols].div(df[base_col], axis=0)
#     return df

# def normalize_cols(df, norm_cols):
#     df_sub = df.copy()
#     df_sub = df_sub[norm_cols]
#     x = df_sub.values #returns a numpy array
#     min_max_scaler = preprocessing.MinMaxScaler()
#     x_scaled = min_max_scaler.fit_transform(x)
#     df_sub=pd.DataFrame(x_scaled, columns=df_sub.columns, index=df_sub.index)
#     df = df.drop(norm_cols, axis=1).join(df_sub)
#     return df

# Load data from sources

In [ ]:
df_get_data = get_latest_file(datapath=datapath, train=train)

# Transform / prepare dataset

In [ ]:
df = df_get_data.copy()
if train:
    df['percentagewmoclienten'] = df['wmoclienten']
    df = df.drop(DROP_COLS, axis=1)
    pl_prepare = make_pipeline(RelativeColumnScaler(dict_relatively_cols=DICT_REALTIVELY_COLS),
                               CustomScaler(cols=LIST_NORM_COLS, scaler=preprocessing.MinMaxScaler()))
    df_prep = pl_prepare.fit_transform(df)
else:
    df = df.drop(DROP_COLS, axis=1)
    pl_prepare = make_pipeline(RelativeColumnScaler(dict_relatively_cols=DICT_REALTIVELY_COLS),
                               CustomScaler(cols=LIST_NORM_COLS, scaler=preprocessing.MinMaxScaler()))
    df_prep = pl_prepare.fit_transform(df)

In [ ]:
df_prep

In [ ]:
# df = df_get_data.copy()
# df['percentagewmoclienten'] = df['wmoclienten']
# df = df.drop(DROP_COLS, axis=1)
# df = make_cols_relatively(df=df, dict_relatively_cols=DICT_REALTIVELY_COLS)
# df = normalize_cols(df=df, norm_cols=LIST_NORM_COLS)

In [ ]:
# TODO

# Write result

In [ ]:
suffix_datetime = datetime.strftime(datetime.now(), format='%Y%m%d%H%M')

if train:
    df_prep.to_parquet(f'../data/df_prep_for_train_WMO_{suffix_datetime}.parquet.gzip',
              compression='gzip')
else:
    df_prep.to_parquet(f'../data/df_prep_for_predict_WMO_{suffix_datetime}.parquet.gzip',
              compression='gzip')

# Appendix
## Code examples to get a subset of the DataFrame based on multiindex

In [ ]:
# Gemeentenaam regels moeten nog gefixt worden
# Subset on columnvalue:
#df_prep[df_prep['gemeentenaam']=='Nijmegen']

# One row / record
df_prep.loc[('WK026801', '2019')]

# Multiple rows / records based on combination of the multiindex
df_prep.loc[[('WK026801', '2018'), ('WK026802', '2018')]]

# Multiple rows / records for one column (works only for series)
#df_prep['gemeentenaam'].loc[(['WK026801', 'WK026802'], ['2018', '2019'])]

# Multiple rows based on both indexes:
df_prep.loc(axis=0)[['WK026801', 'WK026802'], ['2018', '2019']]

# Subset with IndexSlice
idx = pd.IndexSlice

# Subset on one of the multiindex and select a column
#df_prep.loc[idx[:, ['2018', '2019']], idx["gemeentenaam"]]

# Subset on one of the multiindex and select all columns
df_prep.loc[idx['WK026801', :], idx[:]]

In [ ]:
#df_prep[df_prep['gemeentenaam']=='Nijmegen']

## Code to get current versions of loaded packages

In [ ]:
print('\n'.join(f'{m.__name__} {m.__version__}' for m in globals().values() if getattr(m, '__version__', None)))